In [4]:
##This code creates an elementary-level present value and future value “calculator.” 
##Calculates the present value (PV) and future value (FV) for
##discrete forecast periods and multiple growth phases (annuity/growing annuity/perpetuity)
##AI was leveraged in the creation of this code
##Lena Weissman

#defines all calculation functions 
def pv_annuity(C, r, t):
    return C * (1 - (1 + r) ** -t) / r

def pv_perpetuity(C, r):
    return C / r

def fv_annuity(C, r, t):
    return C * (((1 + r) ** t - 1) / r)

def pv_growing_annuity(C, r, g, t):
    return C * (1 - ((1 + g) / (1 + r)) ** t) / (r - g)

def pv_growing_perpetuity(C, r, g):
    return C / (r - g)

def fv_growing_annuity(C, r, g, t):
    if r == g: # Special case when r == g       
        return C * t * (1 + r) ** (t - 1)
    return C * (((1 + r) ** t - (1 + g) ** t) / (r - g))

#Asks user for all of their inputs
calculation_type = input(
    'Would you like to calculate present value or future value? (enter PV or FV): '
).strip().upper()
discount_rate = float(input(
    'What is the discount rate? (enter as a decimal e.g., 0.03 for 3%): '
))

#Allows for distinct payment amounts for each forecast period
discrete_forecast = int(input(
    'How many discrete forecast periods? (enter 0 if none): '
))
discrete_periods = []
if discrete_forecast > 0:
    print("\nEnter cash flows for each discrete period:")
    for i in range(discrete_forecast):
        payment = float(input(f"  Payment for period {i+1}: "))
        length = int(input(f"  Length of this period (in years): "))
        discrete_periods.append({
            'payment': payment,
            'length': length
        })

#Asks user input for growth phases
growth_phases = int(input('\nHow many distinct “growth” phases (enter 0 if none): '))
phases = []  # list to store each growth phase

for i in range(growth_phases):
    print(f"\nPhase {i+1}")
    phase = {}
    phase['payment'] = float(input('  Starting cash flow for this phase?: '))
    phase['rate'] = float(input('  Growth rate? (as decimal, e.g., 0.03): '))

    if i == growth_phases - 1:
        #If it is the last phase ask if it's annuity or perpetuity
        while True:
            typ = input(
                '  Is it an annuity or perpetuity? (enter annuity or perpetuity): '
            ).strip().lower()
            if typ in ['annuity', 'perpetuity']:
                phase['type'] = typ
                break
            else:
                print('Please enter only "annuity" or "perpetuity".')

        if phase['type'] == 'annuity':
            phase['length'] = int(input('  Length of this phase (years)?: '))
        else:
            phase['length'] = 0  #Use 0 for perpetuity
    else:
        phase['type'] = 'annuity'
        phase['length'] = int(input('  Length of this phase (years)?: '))

    phases.append(phase)

total_pv = 0
total_fv = 0
breakdown = []

##PV
if calculation_type == 'PV':
    #PV of discrete forecast periods
    pv_df = 0
    start_time = 1
    for i, period in enumerate(discrete_periods): #Discount each discrete payment back to present
        n = period['length']
        disk_pv = period['payment'] / ((1 + discount_rate) ** start_time)
        pv_df += disk_pv
        start_time += n 

    if discrete_forecast > 0:
        breakdown.append(f"PV Discrete Forecast: ${pv_df:,.2f}")
    total_pv += pv_df

    #PV of each phase discounted back to present
    start_time = sum([p['length'] for p in discrete_periods])
    for idx, phase in enumerate(phases):
        c, g, N, typ = phase['payment'], phase['rate'], phase['length'], phase['type']
        if typ == 'annuity':
            if g == 0:
                pv = pv_annuity(c, discount_rate, N)
            else:
                pv = pv_growing_annuity(c, discount_rate, g, N)
            label = f"PV Phase {idx+1} (Annuity)"
        else:
            if g == 0:
                pv = pv_perpetuity(c, discount_rate)
            else:
                pv = pv_growing_perpetuity(c, discount_rate, g)
            label = f"PV Phase {idx+1} (Perpetuity)"
        pv_deferred = pv / ((1 + discount_rate) ** start_time) #If it does not start at time zero
        breakdown.append(f"{label}: ${pv_deferred:,.2f}")
        total_pv += pv_deferred
        start_time += N

    breakdown.append(f"TOTAL PV: ${total_pv:,.2f}")

##FV
elif calculation_type == 'FV':
    # Calculate full timeline:
    final_time = sum([p['length'] for p in discrete_periods]) + sum([phase['length'] for phase in phases])

    #FV of discrete forecast periods each compounded to the end
    fv_df = 0
    payment_year = 0
    for period in discrete_periods:
        payment = period['payment']
        length = period['length']
        for i in range(length):
            payment_year += 1
            periods_to_compound = final_time - payment_year
            fv = payment * ((1 + discount_rate) ** periods_to_compound)
            fv_df += fv
    if discrete_forecast > 0:
        breakdown.append(f"FV Discrete Forecast: ${fv_df:,.2f}")
    total_fv += fv_df

    #FV of each phase
    phase_start_time = payment_year
    for idx, phase in enumerate(phases):
        c, g, N, typ = phase['payment'], phase['rate'], phase['length'], phase['type']
        phase_end_time = phase_start_time + N
        if typ == 'annuity':
            fv = fv_annuity(c, discount_rate, N) if g == 0 else fv_growing_annuity(c, discount_rate, g, N)
            #Compound from phase end to final_time
            periods_to_compound = final_time - phase_end_time
            fv_deferred = fv * ((1 + discount_rate) ** periods_to_compound)
            label = f"FV Phase {idx+1} (Annuity{' (Growing)' if g != 0 else ''})"
            breakdown.append(f"{label}: ${fv_deferred:,.2f}")
            total_fv += fv_deferred
            phase_start_time += N
        else:
            label = f"FV Phase {idx+1} (Perpetuity)"
            breakdown.append(f"{label}: N/A – FV of perpetuity is not defined.")

    breakdown.append(f"TOTAL FV: ${total_fv:,.2f}")

#Print final breakdown
print("\n> Calculation Breakdown:")
for line in breakdown:
    print("  " + line)

Would you like to calculate present value or future value? (enter PV or FV):  PV
What is the discount rate? (enter as a decimal e.g., 0.03 for 3%):  0.1
How many discrete forecast periods? (enter 0 if none):  2



Enter cash flows for each discrete period:


  Payment for period 1:  1000
  Length of this period (in years):  1
  Payment for period 2:  1000
  Length of this period (in years):  1

How many distinct “growth” phases (enter 0 if none):  3



Phase 1


  Starting cash flow for this phase?:  1100
  Growth rate? (as decimal, e.g., 0.03):  0.05
  Length of this phase (years)?:  3



Phase 2


  Starting cash flow for this phase?:  1100
  Growth rate? (as decimal, e.g., 0.03):  0.02
  Length of this phase (years)?:  10



Phase 3


  Starting cash flow for this phase?:  1500
  Growth rate? (as decimal, e.g., 0.03):  0.02
  Is it an annuity or perpetuity? (enter annuity or perpetuity):  perpetuity



> Calculation Breakdown:
  PV Discrete Forecast: $1,735.54
  PV Phase 1 (Annuity): $2,368.35
  PV Phase 2 (Annuity): $4,525.18
  PV Phase 3 (Perpetuity): $4,488.60
  TOTAL PV: $13,117.66
